<a href="https://colab.research.google.com/github/Qasimrizvi1245/CSProjects/blob/main/algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Imports
We used the `pandas` python library for aggregating and analyzing the student data. We're using the `csv` p

In [ ]:
import pandas as pd
import csv
import random

In [ ]:
roomdata = pd.read_csv("./Data/final_room_data.csv")
residences_table = pd.DataFrame(roomdata)

filtered_residences_table = residences_table[['hall_id', 'hall_name', 'residence_type', 'building_id', 'room_number', 'room_id', 'floor_number', 'floor_plan', 'room_contract_type', 'has_accessibility_ramps', 'price']].copy()
# filtered_residences_table.head(2)

FileNotFoundError: ignored

In [ ]:
sA_df = pd.read_csv("./Data/datasetA_with_roommate_pref.csv")
students = pd.DataFrame(sA_df)
filtered_all_students = students[['name', 'sex', 'RUID', 'student_year', 'accessibility_need', 'low_income_status', 'preferred_residence_type','preferred_hall_ids', 'preferred_price_range', 'max_price']].copy()
print(len(filtered_all_students))

In [ ]:
# Traditional Residence Halls
trad_reshall_table = filtered_residences_table[filtered_residences_table['residence_type'].str.contains('Traditional Residence Hall')]
# trad_reshall_table.head(2)

# Apartments
apts_table = filtered_residences_table[filtered_residences_table['residence_type'].str.contains('Apartment')]

# Suites
suites_table = filtered_residences_table[filtered_residences_table['residence_type'].str.contains('Suite')]

# All Rooms - Singles
singles_table = filtered_residences_table[filtered_residences_table['floor_plan'].str.contains('Traditional Single')]

# All Rooms - Doubles
doubles_table = filtered_residences_table[filtered_residences_table['floor_plan'].str.contains('Traditional Double')]

# Graduate Housing
undergrad_table = filtered_residences_table.loc[filtered_residences_table['room_contract_type'] == "Undergraduate Academic Year (Two Semesters)"]

print(len(filtered_residences_table))

In [ ]:
# move undergrad disability and low income people to new table
undergrad_disability_lowincome_students = pd.DataFrame()
grad_disability_lowincome_students = pd.DataFrame()
accessibility_students = pd.DataFrame()
low_income_students = pd.DataFrame()
undergrad_students = pd.DataFrame()
grad_students = pd.DataFrame()

for index, row in filtered_all_students.iterrows():
  #(CASE 1)
  if (row['student_year'] == "Freshman" or row['student_year'] == "Sophomore" or row['student_year'] == "Junior" or row['student_year'] == "Senior") and row['accessibility_need'] == 1 and row['low_income_status'] == 1:
    undergrad_disability_lowincome_students = undergrad_disability_lowincome_students.append(row)
  #(CASE 2)
  elif row['accessibility_need'] == 1 and row['low_income_status'] == 1:
    grad_disability_lowincome_students = grad_disability_lowincome_students.append(row)
  #(CASE 3)
  elif row['accessibility_need'] == 1:
    accessibility_students = accessibility_students.append(row)
  #(CASE 4) 
  elif row['low_income_status'] == 1:
    low_income_students = low_income_students.append(row)
  #(CASE 5)
  elif (row['student_year'] == "Freshman" or row['student_year'] == "Sophomore" or row['student_year'] == "Junior" or row['student_year'] == "Senior"):
    undergrad_students = undergrad_students.append(row)
  #(CASE 6)
  else: #Graduate
    grad_students = grad_students.append(row)

print(len(undergrad_disability_lowincome_students) + len(grad_disability_lowincome_students) + len(accessibility_students) + len(low_income_students) + len(undergrad_students) + len(grad_students))

# print(undergrad_disability_lowincome_students.head())
# print(grad_disability_lowincome_students.head())
# print(accessibility_students.head())
# print(low_income_students.head())
# print(undergrad_students.head())
# print(grad_students.head())

In [ ]:
# row = grad_disability_lowincome_students.sample()
# print(row['RUID'])

# sa = filtered_residences_table.sample()
# print(sa['price'])

# print(row['max_price'])
# print(len(filtered_residences_table.loc[float(row['max_price']) > filtered_residences_table['price'] ]) > 0)

In [ ]:
assigned_students = pd.DataFrame()

while len(filtered_residences_table) != 0 or len(filtered_all_students) != 0:
  # go through undergrad_disability_lowincome_students
  while len(undergrad_disability_lowincome_students) != 0: #change to while loop with sampling
    row = undergrad_disability_lowincome_students.sample()
    
    if len(filtered_residences_table.loc[(filtered_residences_table['has_accessibility_ramps']) == 1 & (float(row['max_price']) > filtered_residences_table['price'])]) > 0:
      dis_lowinc_rooms = filtered_residences_table.loc[(filtered_residences_table['has_accessibility_ramps']) == 1 & (float(row['max_price']) > filtered_residences_table['price'])]
      assigned_room = dis_lowinc_rooms.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      undergrad_disability_lowincome_students = undergrad_disability_lowincome_students[undergrad_disability_lowincome_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]
    else: #Get regular room
      assigned_room = filtered_residences_table.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      undergrad_disability_lowincome_students = undergrad_disability_lowincome_students[undergrad_disability_lowincome_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]


  
  # Go through grad disability and low income
  while len(grad_disability_lowincome_students) != 0:
    row = grad_disability_lowincome_students.sample()
    if len(filtered_residences_table.loc[(filtered_residences_table['has_accessibility_ramps']) == 1 & (float(row['max_price']) > filtered_residences_table['price'])]) > 0:
      dis_lowinc_rooms = filtered_residences_table.loc[(filtered_residences_table['has_accessibility_ramps']) == 1 & (float(row['max_price']) > filtered_residences_table['price'])]
      assigned_room = dis_lowinc_rooms.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      grad_disability_lowincome_students = grad_disability_lowincome_students[grad_disability_lowincome_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]
    else: #Get regular room
      assigned_room = filtered_residences_table.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      grad_disability_lowincome_students = grad_disability_lowincome_students[grad_disability_lowincome_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]
    

  # Go through accessibility
  while len(accessibility_students) != 0:
    row = accessibility_students.sample()
    if len(filtered_residences_table.loc[(filtered_residences_table['has_accessibility_ramps']) == 1 & (float(row['max_price']) > filtered_residences_table['price'])]) > 0:
      dis_lowinc_rooms = filtered_residences_table.loc[(filtered_residences_table['has_accessibility_ramps']) == 1 & (float(row['max_price']) > filtered_residences_table['price'])]
      assigned_room = dis_lowinc_rooms.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      accessibility_students = accessibility_students[accessibility_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]
    else: #Get regular room
      assigned_room = filtered_residences_table.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      accessibility_students = accessibility_students[accessibility_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]

  # Go through low income
  while len(low_income_students) != 0:
    row = low_income_students.sample()
    if len(filtered_residences_table.loc[(float(row['max_price']) > filtered_residences_table['price'])]) > 0:
      dis_lowinc_rooms = filtered_residences_table.loc[(float(row['max_price']) > filtered_residences_table['price'])]
      assigned_room = dis_lowinc_rooms.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      low_income_students = low_income_students[low_income_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]
    else: #Get regular room
      assigned_room = filtered_residences_table.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      low_income_students = low_income_students[low_income_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]

  # Go through undergrad
  while len(undergrad_students) != 0:
    row = undergrad_students.sample()
    if len(filtered_residences_table.loc[(float(row['max_price']) > filtered_residences_table['price'])]) > 0:
      dis_lowinc_rooms = filtered_residences_table.loc[(float(row['max_price']) > filtered_residences_table['price'])]
      assigned_room = dis_lowinc_rooms.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      undergrad_students = undergrad_students[undergrad_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]
    else: #Get regular room
      assigned_room = filtered_residences_table.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      undergrad_students = undergrad_students[undergrad_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]

  # Go through grad
  while len(grad_students) != 0:
    row = grad_students.sample()
    if len(filtered_residences_table.loc[(float(row['max_price']) > filtered_residences_table['price'])]) > 0:
      dis_lowinc_rooms = filtered_residences_table.loc[(float(row['max_price']) > filtered_residences_table['price'])]
      assigned_room = dis_lowinc_rooms.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      grad_students = grad_students[grad_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]
    else: #Get regular room
      assigned_room = filtered_residences_table.head(1)
      # print(assigned_room['room_id'])
      tempdf = pd.DataFrame({'RUID': int(row['RUID']),
                            'hall_id': assigned_room['hall_id'],
                            'building_id': assigned_room['building_id'],
                            'room_id': assigned_room['room_id']})
      assigned_students = assigned_students.append(tempdf, ignore_index=True)
      # Dropping from tables
      grad_students = grad_students[grad_students['RUID'] != float(row['RUID'])]
      filtered_all_students = filtered_all_students[filtered_all_students['RUID'] != float(row['RUID'])]
      filtered_residences_table = filtered_residences_table[~(filtered_residences_table['room_id'].isin(assigned_room['room_id']) & filtered_residences_table['hall_id'].isin(assigned_room['hall_id']) & filtered_residences_table['building_id'].isin(assigned_room['building_id']))]

print(len(filtered_all_students))
print(len(filtered_residences_table))

In [ ]:
# Print to csv

assigned_students.to_csv("assigned_students.csv", index=False)